In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from armscan_env import config
from armscan_env.volumes.loading import load_sitk_volumes, resize_sitk_volume

config = config.get_config()

In [ ]:
volumes = load_sitk_volumes(normalize=False)

In [ ]:
normalized_volumes = resize_sitk_volume(volumes)

In [ ]:
array = sitk.GetArrayFromImage(normalized_volumes[1])
plt.imshow(array[40, :, :])
print(f"Slice value range: {np.min(array)} - {np.max(array)}")

In [ ]:
array = sitk.GetArrayFromImage(volumes[0])
plt.imshow(array[40, :, :])
print(f"Slice value range: {np.min(array)} - {np.max(array)}")